### Хи-квадрат Пирсона (Chi-squared)
**Исследование взаимосвязи между двумя номинативными переменными**

Хи-квадрат Пирсона это непараметрический метод, который позволяет оценить значимость различий между фактическим
(выявленным в результате исследования) количеством исходов или качественных характеристик выборки,
попадающих в каждую категорию, и теоретическим количеством, которое можно ожидать в изучаемых группах при справедливости нулевой гипотезы.
Выражаясь проще, метод позволяет оценить статистическую значимость различий двух или нескольких относительных показателей (частот, долей).

**Основные шаги**
1. Сформулировать задачу, H0 и HA
2. Провести разведывательный анализ данных, чтобы убедится в том, что данные отвечают требованиям
3. Получить значения chi2 (показывает силу отклонения), P-value (вероятность отклонить верную H0), degrees of freedom (нужны для расчета критического значения chi2), критическое значение chi2 (сравнив фактическое можно сделать вывод о значимости влияния).

<a href="https://lit-review.ru/biostatistika/kriterijj-khi-kvadrat-pirsona/">Последовательность действий подробнее</a>

** Требования к данным:**
1. Номинативные переменные с бинарными или порядковыми значениями. Более точный результат будет получен для бинарных номинативных переменных.
2. Независимость и случайность выборок
3. Общее количество наблюдений было более 20. С увеличением объема выборки точность критерия повышается;
4. Теоретическая (ожидаемая) частота для каждого выборочного интервала (соответствующая нулевой
гипотезе) должна, быть **более 5**. Если ожидаемое явление принимает значение менее 5, то необходимо
использовать точный критерий Фишера;
5. Если теоретическая (ожидаемая) частота в случае использования четырехпольных таблиц (2х2) принимает
значение **менее 10 (а именно 5<x<10)**, необходим расчет поправки Йетса;
6. Сравниваемые частоты должны быть примерно **одного размера**;


<a href="https://www.statology.org/correlation-between-categorical-variables/">Тесты для датасетов, которые не отвечают критериям</a>

In [19]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats
from scipy.stats import chi2_contingency

### Постановка задачи
Провести исследование взаимосвязи двух номинативных переменных Cargo count и Passengers count

In [20]:
df = pd.read_csv('/Users/mitya/Desktop/cargo_passengers.csv')
df.head()

,Cargo count,Passengers count
0,0,0
1,2,2
2,2,2
3,0,0
4,1,1


In [21]:
table = sm.stats.Table.from_data(df[['Cargo count', 'Passengers count']])
contingency_table = table.table_orig
contingency_table
# Важно! Хи-квадрат Пирсона не поодходит для датасета т.к. есть экстремально низкие значения > 5

Passengers count,0,1,2,3
Cargo count,,,,
0,1555,11,23,4
1,146,538,9,1
2,273,43,1077,2
3,16,1,4,72
4,14,5,19,0


In [22]:
fit_val_table = table.fittedvalues
fit_val_table

Passengers count,0,1,2,3
Cargo count,,,,
0,837.123902,249.800446,472.866396,33.209257
1,364.698046,108.827062,206.007080,14.467812
2,733.074603,218.751803,414.092041,29.081552
3,48.871640,14.583454,27.606136,1.938770
4,20.231808,6.037236,11.428347,0.802609


In [23]:
resid_table = table.resid_pearson
resid_table

Passengers count,0,1,2,3
Cargo count,,,,
0,24.811592,-15.109098,-20.687801,-5.068634
1,-11.451917,41.139961,-13.725898,-3.540753
2,-16.992379,-11.882942,32.576528,-5.021862
3,-4.702111,-3.556968,-4.492852,50.316983
4,-1.385469,-0.422142,2.239747,-0.337777


In [24]:
chi2, df, p_value = table.test_nominal_association().statistic, table.test_nominal_association().df, table.test_nominal_association().pvalue
print('chi2', chi2)
print('p_value', p_value)
print('df', df)

chi2 7432.525215499285
p_value 0.0
df 12


In [25]:
chi2, p_val, dof, exp = chi2_contingency(contingency_table, correction=True)
print('chi2', chi2)
print('p_value', p_val)
print('df', dof)

chi2 7448.762084141021
p_value 0.0
df 12


In [26]:
critical_value = scipy.stats.chi2.ppf(1-.001, df=df)
critical_value

32.90949040736021

In [27]:
# Отклоняем H0. Фактическое значение chi2 значительно превышает критическое значение chi2 при α = 0.1% и df=32
# Помним, что Хи-квадрат не подходит для датасета, возможны некорректные результаты теста.

In [28]:
# Cramer’s V (коэффициент корреляции двух номинативных переменных, который применяют для таблиц более 2x2) принимает значение от 0 до 1
# Interpretation of effect size
# ES ≤ 0.2 The result is weak. Although the result is statistically significant, the fields are only weakly associated.
# 0.2 < ES ≤ 0.6 The result is moderate. The fields are moderately associated.
# ES > 0.6 The result is strong. The fields are strongly associated.
contingency_table = np.array(contingency_table)
N = np.sum(contingency_table)
minimum_dimension = min(contingency_table.shape)-1

# Calculate Cramer's V
result = np.sqrt((chi2 / N) / minimum_dimension)
result

0.8069525930731775

In [29]:
# Отклоняем H0

### Постановка задачи
Провести исследование взаимосвязи двух номинативных переменных Passengers и Individual

In [30]:
df = pd.read_csv('/Users/mitya/Desktop/passengers_individual.csv')
df.head()

,Passengers,Individual calc
0,0,0
1,2,0
2,2,0
3,0,0
4,1,0


In [31]:
table = sm.stats.Table.from_data(df[['Individual calc', 'Passengers']])
contingency_table = table.table_orig
contingency_table

Passengers,0,1,2,3
Individual calc,,,,
0,1017,212,333,19
1,987,386,799,60


In [32]:
fit_val_table = table.fittedvalues
fit_val_table

Passengers,0,1,2,3
Individual calc,,,,
0,830.926829,247.95122,469.365854,32.756098
1,1173.073171,350.04878,662.634146,46.243902


In [33]:
chi2, df, p_value = table.test_nominal_association().statistic, table.test_nominal_association().df, table.test_nominal_association().pvalue
print('chi2', chi2)
print('p_value', p_value)
print('df', df)

chi2 157.6389950100305
p_value 0.0
df 3


In [34]:
chi2, p_val, dof, exp = chi2_contingency(contingency_table, correction=False)
print('chi2', chi2)
print('p_value', p_val)
print('df', dof)

chi2 157.63899501003044
p_value 5.924180789938196e-34
df 3


In [35]:
critical_value = scipy.stats.chi2.ppf(1-.001, df=df)
critical_value

16.26623619623813

In [36]:
# Отклоняем H0. Фактическое значение chi2 значительно превышает критическое значение chi2 при α = 0.1% и df=3
# Помним, что Хи-квадрат не подходит для датасета, возможны некорректные результаты теста.

In [37]:
# Cramer’s V (коэффициент корреляции двух номинативных переменных, который применяют для таблиц более 2x2) принимает значение от 0 до 1
# Interpretation of effect size
# ES ≤ 0.2 The result is weak. Although the result is statistically significant, the fields are only weakly associated.
# 0.2 < ES ≤ 0.6 The result is moderate. The fields are moderately associated.
# ES > 0.6 The result is strong. The fields are strongly associated.
contingency_table = np.array(contingency_table)
N = np.sum(contingency_table)
minimum_dimension = min(contingency_table.shape)-1

# Calculate Cramer's V
result = np.sqrt((chi2 / N) / minimum_dimension)
result

0.20332857927185197

In [38]:
# Отклоняем H0